<a href="https://colab.research.google.com/github/SagarBajaj14/AI-chatbot/blob/main/AIchatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 5.2 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=81da44a891f2fae3f9cd2de2f48362ade901c34bbd0aebb8197df9f4cc50d936
  Stored in directory: /root/.cache/pip/wheels/65/9b/15/cb1e6b279c14ed897530d15cfd7da8e3df8a947e593f5cfe59
Successfully built tflearn


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
import tflearn
import tensorflow
import random
import json
import pickle


with open("intents.json") as file:
    data = json.load(file)

# opening a saved model if it exists
try:
  with open("data.pickle", "rb") as f:
    words, labels, training, output = pickle.load(f)

#preprocessing the data for the model
# input will be a list of bag of words and the output consists of labels/tags from intents.json file
except:
  words = []  # stemmed words from the data
  labels= []  # tags from the data
  docs_x = [] # training data that will be processed and compared to words list
  docs_y = [] # tags corresponding to each word in data

  for intent in data["intents"]:
      for pattern in intent["patterns"]:
          wrds = nltk.word_tokenize(pattern)  
          words.extend(wrds)
          docs_x.append(wrds)
          docs_y.append(intent["tag"])


      if intent["tag"] not in labels:
          labels.append(intent['tag'])
      
  words  = [stemmer.stem(w.lower()) for w in words if w != "?"]
  words = sorted(list(set(words)))

  labels = sorted(labels)

  training = []
  output = []

  out_empty = [0 for _ in range(len(labels))]

  for x, doc in enumerate(docs_x):
    bow = []
    wrds = [stemmer.stem(w.lower()) for w in doc]
    for w in words:
      if w in wrds:
        bow.append(1)
      else:
        bow.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1 # tag output 

    training.append(bow)
    output.append(output_row)

  training = np.array(training)
  output = np.array(output)

  with open("data.pickle", "rb") as f:
    pickle.dump((words, labels, training, output), f)

  #print(training)
  #print(output)



[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0]]


In [5]:
# Neural network for the model

# neural net consists of input layer as bag of words two hidden layers and output layer as tags predicted
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

try:
  model.load("model.tflearn") #load model if saved 
except:
  model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True) # fit the model and then save
  model.save("model.tflearn")


Training Step: 4999  | total loss: 0.02109 | time: 0.027s
| Adam | epoch: 1000 | loss: 0.02109 - acc: 1.0000 -- iter: 32/39
Training Step: 5000  | total loss: 0.02195 | time: 0.034s
| Adam | epoch: 1000 | loss: 0.02195 - acc: 1.0000 -- iter: 39/39
--


In [13]:
def bag_of_words(s, words):
  bag = [0 for _ in range(len(words))]

  s_words = nltk.word_tokenize(s)
  s_words = [stemmer.stem(word.lower()) for word in s_words]

  for sen in s_words:
    for i,w in enumerate(words):
      if w==sen:
        bag[i] = 1

  return np.array(bag)

def chat():
  print("Start talking with the chatbot(press q to quit)")
  while True:
    inp = input("User: ")
    if inp.lower() == "q":
      break

    res = model.predict([bag_of_words(inp, words)])[0]
    # choose the maximum probability tag from the result.
    res_index = np.argmax(res)   
    tag = labels[res_index]

    # Check whether predicted probability is above 80%  
    if res[res_index] > 0.8:
      for tags in data["intents"]:
        if tags["tag"] == tag:
          responses = tags["responses"]

      print(random.choice(responses))
    else:
      print("I am not sure what you are talking about please try again!")

chat()

Start talking with the chatbot(press q to quit)
User: Hey how are you
Hey there, how can I help you?
User: what is ur name
I am Alexa
User: How old are you
I am 20!
User: can i have the menu
Today's special is cheesy fries and oreo shake
User: i would like to order that
Thanks for your order it will take 10 minutes to get it ready
User: what are ur timings
I am not sure what you are talking about please try again!
User: what are the shop timings
You are welcome in our shop from Monday to Sunday 10am-9pm
User: See you soon
See you soon
User: Goodbye
Goodbye!
User: q
